# 4. Graph RAG
<img src="https://bluetick-website-images.bluetickconsultants.com/images/blog/rag-graphrag/key-differentiators-of-graphrag.webp" height=300px>

In [102]:
!pip install --upgrade --quiet  langchain langchain-community langchain-groq neo4j

In [103]:
!pip install --upgrade --quiet langchain_experimental

### Importing libraries

In [104]:
import os
from langchain_community.graphs import Neo4jGraph
from langchain_groq import ChatGroq
from langchain_core.documents import Document
import fitz
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain.chains import GraphCypherQAChain

In [105]:
## Graphdb configuration
NEO4J_URI=""
NEO4J_USERNAME=""
NEO4J_PASSWORD=""

In [106]:

os.environ["NEO4J_URI"]=NEO4J_URI
os.environ["NEO4J_USERNAME"]=NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"]=NEO4J_PASSWORD

In [107]:
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

In [108]:
graph

In [109]:
groq_api_key=""

In [110]:

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001E232DF0440>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E232DF36E0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [111]:


def read_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    # read only till 4 pages
    
    for page_num in range(2):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

pdf_path = "pdf/IPCC_AR6_SYR_SPM.pdf" 
text = read_pdf(pdf_path)

documents = [Document(page_content=text)]
print(documents)


[Document(metadata={}, page_content='A Report of the Intergovernmental Panel on Climate Change\nCLIMATE CHANGE 2023\nSynthesis Report\nSummary for Policymakers\n')]


In [112]:

llm_transformer=LLMGraphTransformer(llm=llm)

In [113]:
graph_documents=llm_transformer.convert_to_graph_documents(documents)

In [114]:
documents

[Document(metadata={}, page_content='A Report of the Intergovernmental Panel on Climate Change\nCLIMATE CHANGE 2023\nSynthesis Report\nSummary for Policymakers\n')]

In [115]:
graph_documents

[GraphDocument(nodes=[Node(id='Climate Change 2023', type='Report', properties={}), Node(id='Synthesis Report', type='Report_part', properties={}), Node(id='Summary For Policymakers', type='Report_part', properties={}), Node(id='Intergovernmental Panel On Climate Change', type='Organization', properties={})], relationships=[Relationship(source=Node(id='Climate Change 2023', type='Report', properties={}), target=Node(id='Synthesis Report', type='Report_part', properties={}), type='CONTAINS', properties={}), Relationship(source=Node(id='Climate Change 2023', type='Report', properties={}), target=Node(id='Summary For Policymakers', type='Report_part', properties={}), type='CONTAINS', properties={}), Relationship(source=Node(id='Intergovernmental Panel On Climate Change', type='Organization', properties={}), target=Node(id='Climate Change 2023', type='Report', properties={}), type='CREATED', properties={})], source=Document(metadata={}, page_content='A Report of the Intergovernmental Panel

In [116]:
graph_documents[0].nodes

[Node(id='Climate Change 2023', type='Report', properties={}),
 Node(id='Synthesis Report', type='Report_part', properties={}),
 Node(id='Summary For Policymakers', type='Report_part', properties={}),
 Node(id='Intergovernmental Panel On Climate Change', type='Organization', properties={})]

In [117]:
graph_documents[0].relationships

[Relationship(source=Node(id='Climate Change 2023', type='Report', properties={}), target=Node(id='Synthesis Report', type='Report_part', properties={}), type='CONTAINS', properties={}),
 Relationship(source=Node(id='Climate Change 2023', type='Report', properties={}), target=Node(id='Summary For Policymakers', type='Report_part', properties={}), type='CONTAINS', properties={}),
 Relationship(source=Node(id='Intergovernmental Panel On Climate Change', type='Organization', properties={}), target=Node(id='Climate Change 2023', type='Report', properties={}), type='CREATED', properties={})]

In [118]:
graph_documents

[GraphDocument(nodes=[Node(id='Climate Change 2023', type='Report', properties={}), Node(id='Synthesis Report', type='Report_part', properties={}), Node(id='Summary For Policymakers', type='Report_part', properties={}), Node(id='Intergovernmental Panel On Climate Change', type='Organization', properties={})], relationships=[Relationship(source=Node(id='Climate Change 2023', type='Report', properties={}), target=Node(id='Synthesis Report', type='Report_part', properties={}), type='CONTAINS', properties={}), Relationship(source=Node(id='Climate Change 2023', type='Report', properties={}), target=Node(id='Summary For Policymakers', type='Report_part', properties={}), type='CONTAINS', properties={}), Relationship(source=Node(id='Intergovernmental Panel On Climate Change', type='Organization', properties={}), target=Node(id='Climate Change 2023', type='Report', properties={}), type='CREATED', properties={})], source=Document(metadata={}, page_content='A Report of the Intergovernmental Panel

In [119]:
chain=GraphCypherQAChain.from_llm(llm=llm,graph=graph,verbose=True,allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x000001E233028740>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001E232DF0440>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E232DF36E0>, model_name=

In [120]:
response=chain.invoke({"query":"What are some climate related risks?"})

response




> Entering new GraphCypherQAChain chain...


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: AFFECTS)} {position: line: 2, column: 26, offset: 32} for query: 'cypher\nMATCH (c:ClimateRisk)<-[:AFFECTS]-(r:Risk)\nRETURN r.name\n'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing la

Generated Cypher:
cypher
MATCH (c:ClimateRisk)<-[:AFFECTS]-(r:Risk)
RETURN r.name

Full Context:
[]

> Finished chain.


{'query': 'What are some climate related risks?',
 'result': "I don't know the answer. \n"}